# Data Integration Courselet

**Authors:** 
 - Siddarth Vijayakumar Sivakala (Z2061678@students.niu.edu)
 - Manvitha Kuncham (Z1959412@students.niu.edu)
 - David Koop (dakoop@niu.edu)

**Last Updated:** 2025-10-27

This courselet provides information on using polars to do data integration. Data integration is the process of combining data from multiple sources into a single, unified view. This generally involves bringing together data from different databases, file formats, and data streams to create a more comprehensive dataset to answer complex questions. A key challenge in data integration is dealing with inconsistencies and discrepancies between different data sources, including differences in data formats, naming conventions, and conflicting data values. We will be using datasets that measure the economic impact of world travel on different countries. For this data integration process we will be using data from the following sources:

 * [United Nations World Tourism Organization](https://www.unwto.org) (UNWTO),
 * [World Bank](https://data.worldbank.org) (WB),
 * [OECD](https://data.oecd.org) 
 * [United Nations World Population Division](https://population.un.org/wpp/) (UNPOP)

 These datasets are have been partially wrangled and are available from GitHub:

* <https://raw.githubusercontent.com/dakoop/fount-data-integration/refs/heads/main/unwto.parquet>
* <https://raw.githubusercontent.com/dakoop/fount-data-integration/refs/heads/main/wb.parquet>
* <https://raw.githubusercontent.com/dakoop/fount-data-integration/refs/heads/main/oecd.parquet>
* <https://raw.githubusercontent.com/dakoop/fount-data-integration/refs/heads/main/population.parquet>

 In the World Bank dataset, `ST.INT.RCPT.CD` are the receipts and `ST.INT.XPND.CD` are the expenditures. In the UNWTO dataset, "Inbound Tourism expenditure in the country" are the receipts and "Outbound Tourism expenditure in other countries" are the expenditures. In the OECD dataset, "Total international expenditure" and "Total international receipts" are the columns for expenditures and receipts, respectively. Other attributes should be decipherable.


In [ ]:
%config InteractiveShell.ast_node_interactivity = 'last_expr_or_assign'

In [ ]:
import polars as pl

In [ ]:
unwto = pl.read_parquet(
    "https://raw.githubusercontent.com/dakoop/fount-data-integration/refs/heads/main/unwto.parquet"
)

In [ ]:
wb = pl.read_parquet(
    "https://raw.githubusercontent.com/dakoop/fount-data-integration/refs/heads/main/wb.parquet"
)

In [ ]:
oecd = pl.read_parquet(
    "https://raw.githubusercontent.com/dakoop/fount-data-integration/refs/heads/main/oecd.parquet"
)

In [ ]:
pop = pl.read_parquet(
    "https://raw.githubusercontent.com/dakoop/fount-data-integration/refs/heads/main/population.parquet"
)

### Data Cleaning

Now that we have gathered all the required data from different sources, its essential to clean the data before we jump into processing the information. Here lets not jump into removing the null values for now, but notice how the OECD dataset `oecd` is filled with `..` values instead of null values. Let's clean that up.

In [ ]:
oecd_updated = oecd.with_columns(pl.col("value").replace("..", None).cast(pl.Float64))

### Data Transformation

Now, observe that this dataset is not tidy. The dataset records information for the same country and year in different rows; the variables are listed in separate rows instead of columns. We can use the `pivot` operation to fix this.

In [ ]:
oecd_pivoted = oecd_updated.pivot(on="Variable", values="value")

There is a similar problem with the UNWTO dataset.

In [ ]:
unwto

In [ ]:
unwto_pivoted = unwto.pivot(on="Variable", values="value")

After pivoting the data, we see six columns yet many of those cells are missing values. We are intrested the total expenditures inbound and outbound. You may notice that when we have complete data for inbound or outbound, the other two columns (travel and transport) add up to the total expenditure. We can use the coalesce operation to fill in missing totals.

In [ ]:
def add_if_not_both_null(col1, col2):
    return (
        pl.when(col1.is_not_null() | col2.is_not_null())
        .then(col1.fill_null(0) + col2.fill_null(0))
        .otherwise(None)
    )


unwto_filled = unwto_pivoted.with_columns(
    pl.coalesce(
        pl.col("Inbound Tourism expenditure in the country"),
        add_if_not_both_null(
            pl.col("Inbound Travel"), pl.col("Inbound Passenger transport")
        ),
    ),
    pl.coalesce(
        pl.col("Outbound Tourism expenditure in other countries"),
        add_if_not_both_null(
            pl.col("Outbound Travel"), pl.col("Outbound Passenger transport")
        ),
    ),
)

Finally, we have a similar transformation for the World Bank data.

In [ ]:
wb_pivoted = wb.pivot(on="Indicator Code", values="value")

### Schema Matching

Next, we are going to align the schemas of these datasets. This involves renaming columns so that they have consistent names across datasets, selecting relevant columns to keep, and ensuring that data types are consistent. This involves identifying which attributes refer to the same thing or concept and how they are related to each other. We want our shared schema to have the following columns:

* `countryCodeNum`: the numeric code for the country
* `countryCodeAlpha`: the alphabetic code for the country
* `country`: the name of the country
* `year`: the year the amounts were recorded
* `receipts`: the amount of money (in millions of USD) spent in the country on travel
* `expenditures`: the amount of money (in millions of USD) residents spend in other countries while on travel
* `source_dataset`: the corresponding data source `("unwto", "wb", "oecd")`

We will rename the columns in each dataset to match this schema, and drop any columns that are not needed.

In [ ]:
unwto_renamed = (
    unwto_filled.rename(
        {
            "Inbound Tourism expenditure in the country": "receipts",
            "Outbound Tourism expenditure in other countries": "expenditures",
            "CountryCode": "countryCodeNum",
            "Country": "country",
            "Year": "year",
        }
    )
    .with_columns(pl.lit("unwto").alias("source_dataset"))
    .select(
        "country",
        "countryCodeNum",
        "year",
        "receipts",
        "expenditures",
        "source_dataset",
    )
)

Similarly, we can do this for the WB dataset. In addition, this dataset has duplicate rows and its values are specified in US dollars instead of millions of US dollars.

In [ ]:
wb_renamed = (
    wb_pivoted.rename(
        {
            "ST.INT.RCPT.CD": "receipts",
            "ST.INT.XPND.CD": "expenditures",
            "Country Code": "countryCodeAlpha",
            "Country Name": "country",
            "Year": "year",
        }
    )
    .with_columns(
        pl.col("receipts", "expenditures") / 1e6, pl.lit("wb").alias("source_dataset")
    )
    .unique(subset=["country", "year"])
    .drop("Indicator Name")
)

##### Exercise

Complete the schema matching for the OECD dataset.

##### Solution

In [ ]:
oecd_renamed = (
    oecd_pivoted.rename(
        {
            "Total international receipts": "receipts",
            "Total international expenditure": "expenditures",
            "Country": "country",
            "Year": "year",
        }
    )
    .with_columns(pl.lit("oecd").alias("source_dataset"))
    .select("country", "year", "receipts", "expenditures", "source_dataset")
)


### Data Integration

While our main goal is to integrate these datasets into a single dataset, we can also integrate data by merging two datasets with **different** information. For example, we can integrate the UNPOP dataset with the UNWTO dataset to get population information for each country and year. In this case, we can use the numeric code for each country along with the year to join the datasets.

In [ ]:
pop

In [ ]:
unwto_renamed.join(
    pop.select("NumericCode", "Year", "Population-Thousands").rename(
        {
            "NumericCode": "countryCodeNum",
            "Year": "year",
            "Population-Thousands": "population",
        }
    ),
    on=["countryCodeNum", "year"],
)

You may have noticed that we lost some rows when we joined the population data. This is because some country codes in the UNWTO dataset did not have a matching country code in the UNPOP dataset. To avoid losing data, we can perform a **left** join instead, which keeps all rows from the UNWTO dataset and adds population data where available.

In [ ]:
unwto_renamed.join(
    pop.select("NumericCode", "Year", "Population-Thousands").rename(
        {
            "NumericCode": "countryCodeNum",
            "Year": "year",
            "Population-Thousands": "population",
        }
    ),
    on=["countryCodeNum", "year"],
    how="left",
)

Polars also supports right joins and outer joins. A right join keeps all rows from the right dataframe, while an outer join keeps all rows from both dataframes. In addition, it support an anti-join which tells us which rows in the left dataframe do not have a match in the right dataframe. Thus, we can find those countries for which the country codes did not match.

In [ ]:
unwto_renamed.join(
    pop.select("NumericCode", "Year", "Population-Thousands").rename(
        {
            "NumericCode": "countryCodeNum",
            "Year": "year",
            "Population-Thousands": "population",
        }
    ),
    on=["countryCodeNum", "year"],
    how="anti",
)

### Entity Resolution

We will revisit the integration of population data after fusing the three datasets together. For now, we want to use this population data for its country code mapping information. The UNPOP dataset has both the numeric and alphabetic country codes, along with the country name. This will allow us to identify the same countries across datasets. Let's create a dataframe with these codes.

In [ ]:
country_codes = (
    pop.select(
        pl.col("Country").str.to_uppercase(), pl.col("NumericCode"), pl.col("AlphaCode")
    )
    .rename(
        {
            "Country": "country",
            "AlphaCode": "countryCodeAlpha",
            "NumericCode": "countryCodeNum",
        }
    )
    .unique()
)

Now we try to resolve both the UNWTO and WB dataframe with the available information. UNWTO has numeric country codes while WB uses the alphabetic country codes. We will again use left joins to add the missing country codes to each dataset.

In [ ]:
unwto_resolved = unwto_renamed.join(country_codes, on="countryCodeNum", how="left")

In [ ]:
wb_resolved = wb_renamed.join(country_codes, on="countryCodeAlpha", how="left")

Now, even though the UNWTO and WB used different codes, our country code mapping allows us to align both datasets to use the same codes.

In [ ]:
unwto_resolved.drop("country_right").join(
    wb_resolved.drop("country_right"), on=["countryCodeNum", "year"]
)

While this works, there are a few countries that we were not able to match. Let's look at the OECD dataset now. Here, we have neither of the codes, only the country name.

In [ ]:
oecd_resolved = oecd_renamed.join(
    country_codes,
    on="country",
    how="left",
)

That did not work out well. There were no matches! If we look back at the codes, we might notice that all of the country names in our country codes mapping are in uppercase while the OECD dataset has mixed case country names. Let's convert the country names to uppercase in both datasets and try again.

In [ ]:
oecd_resolved = oecd_renamed.with_columns(pl.col("country").str.to_uppercase()).join(
    country_codes,
    on="country",
    how="left",
)

We can look at the countries that did not match using the anti-join.

In [ ]:
(
    oecd_renamed.with_columns(pl.col("country").str.to_uppercase())
    .join(
        country_codes,
        on="country",
        how="anti",
    )["country"]
    .unique()
)

We can see that most of the countries do appear in the country codes mapping, but there are some discrepancies in naming conventions. For example, "CZECH REPUBLIC" in OECD is "CZECHIA" in the country codes mapping. For this handful of countries, we could manually create a mapping to fix these discrepancies.

##### Exercise

Create a mapping for the unmatched countries from the names in the OECD dataset to the corresponding names in the country codes mapping. Create a dictionary called `oecd_mapping` with these mappings.

##### Solution

In [ ]:
to_find = ["KOREA", "RUSSIA", "CZECH", "UNITED STATES", "CHINA", "SLOVAK"]
with pl.Config(tbl_rows=20, fmt_str_lengths=100):
    display(country_codes.filter(pl.col("country").str.contains("|".join(to_find))))

In [ ]:
oecd_mapping = {
    "REPUBLIC OF KOREA": "KOREA",
    "RUSSIAN FEDERATION": "RUSSIA",
    "CZECHIA": "CZECH REPUBLIC",
    "UNITED STATES OF AMERICA": "UNITED STATES",
    "CHINA": "CHINA (PEOPLE'S REPUBLIC OF)",
    "SLOVAKIA": "SLOVAK REPUBLIC",
}

### Manual Entity Resolution

Now, we can use this manual mapping to add country codes to the OECD dataset. We can match everything that matches exactly and then use the manual mappings for the rest. The coalesce operation will use the second mapping when original country code is null.

In [ ]:
oecd_manual = (
    oecd_resolved.join(
        country_codes.with_columns(pl.col("country").replace(oecd_mapping)),
        on="country",
        how="left",
    )
    .with_columns(
        pl.coalesce(pl.col("countryCodeAlpha"), pl.col("countryCodeAlpha_right")),
        pl.coalesce(pl.col("countryCodeNum"), pl.col("countryCodeNum_right")),
    )
    .drop("countryCodeNum_right", "countryCodeAlpha_right")
)

### Probablistic Entity Resolution

Recall that we had some missing matches in both the UNWTO and WB datasets when we tried to add country codes. In addition, the manual entity resolution may also not scale well, especially when entity names are harder to match. In these cases, we can use probablistic entity resolution techniques to find likely matches based on similarity. Polars does not have built-in support for this, but we can use the `splink` library to compute similarity scores and find the best matches.

[**Splink**](https://moj-analytical-services.github.io/splink/index.html) is a python package that does probabilistic entity resolution in python using DuckDB. You can install it via `pip install splink` or `conda install splink` or similar commands with other tools. It also supports Spark and other big data frameworks. While splink is very powerful, it is also quite complex. We will step through some of its features, but for more details, please refer to the [documentation](https://moj-analytical-services.github.io/splink/index.html).

One tool that splink provides is the ability to analyze where data is missing using completeness charts. These charts show the percentage of non-missing values for each column in the dataset. In our case, this shows that our attempt to resolve the entities with only country codes worked well but not perfectly. 

In [ ]:
from splink import DuckDBAPI
from splink.exploratory import completeness_chart

completeness_chart(unwto_resolved, db_api=DuckDBAPI())

In [ ]:
completeness_chart(wb_resolved, db_api=DuckDBAPI())

In [ ]:
completeness_chart(oecd_resolved, db_api=DuckDBAPI())

TWe can see that all three datasets country codes or names that do not link to the population data. (We returned to the OECD dataset before manual resoltuion.) Spink offers two modes of operation: **linking** and **deduplication**. The main difference is that linking will only link **between** two dataframes while deduplication will link **within** a single dataframe. Let's look at linking first. Splink will track rows by a unique index so we will add this to each dataframe using the `with_row_count` method. We will also convert the WB country names to uppercase.

In [ ]:
wb_w_ind = (
    wb_resolved.with_row_index("unique_id")
    .with_columns(pl.col("country").str.to_uppercase())
    .drop(["country_right"])
)

In [ ]:
unwto_w_ind = unwto_resolved.with_row_index("unique_id").drop(["country_right"])

In [ ]:
oecd_w_ind = oecd_resolved.with_row_index("unique_id")

Now, we need to set up the splink settings for linking. Without this configuration, Splink will not know which fields to compare, how to compare them, and what should count as a potential match. Specifically, we need the following settings:

* **Link Type** (`link_type`): Whether to compare records across datasets (`link_only`), within a dataset (`dedupe_only`), or both (`link_and_dedupe`).
* **Blocking Rules** (`blocking_rules_to_generate_predictions`): Criteria to limit comparisons to likely matches, improving efficiency. These rules define which pairs of records should be compared so that we do not have to compare every record in one dataset to every record in the other dataset.
* **Comparisons** (`comparisons`): How to compare specific fields. This includes the functions to use as well as how to define different levels of agreement (e.g. exact match, partial match, null level).

There are a number of other settings that we can also specify, but we will start with these. Second, we need to create a linker object that identifies the datasets to link and the settings to use. You can also specify which database API to use; we will use DuckDB.

In [ ]:
import splink.comparison_library as cl
from splink import DuckDBAPI, Linker, SettingsCreator, block_on

settings = SettingsCreator(
    link_type="link_only",
    comparisons=[
        cl.JaroWinklerAtThresholds(
            "country",
            [0.9, 0.8, 0.7],
        ),
        cl.ExactMatch("year"),
        # cl.ExactMatch("countryCodeAlpha"),
        # cl.ExactMatch("countryCodeNum"),
    ],
    blocking_rules_to_generate_predictions=[block_on("year")],
    # blocking_rules_to_generate_predictions=[
    #     "substr(l.Country,1,3) = substr(r.Country,1,3) AND l.Year = r.Year",
    # ],
)

db_api = DuckDBAPI()
linker = Linker(
    [wb_w_ind, oecd_w_ind, unwto_w_ind],
    settings,
    db_api=db_api,
    input_table_aliases=["wb", "oecd", "unwto"],
)

Now, we need to estimate the parameters that the linker will use to predict which rows match. The estimate of u measures how likely we are to match by coincidence even though the entities should not be matched. The estimate of m tells us how likely we are to match when the entities should be matched. We can estimate u using random sampling, and then use the expectation maximization (EM) algorithm to estimate m. Note that we will use the codes to help the model learn the types of country name matches we might expect.

The conceptual process involves a sequential Expectation-Maximization (EM) approach for learning matching parameters. The year EM step, with a blocking scope of "year" (block_on("year")), estimates initial parameters using only pairs that share the same Year. The Second EM step then refines these parameters, using a data defined by matching codes, to further optimize $m$, $u$, and $lambda$ within that specific same-country context. Think of it like fine-tuning a model

* You train on one subset (year blocks) to get good general structure.
* Then you fine-tune on another subset (country code blocks) to adapt parameters.

In [ ]:
linker.training.estimate_u_using_random_sampling(max_pairs=1e7)
linker.training.estimate_parameters_using_expectation_maximisation(block_on("year"))
linker.training.estimate_parameters_using_expectation_maximisation(
    block_on("countryCodeAlpha"), block_on("countryCodeNum")
)

Now, we use the predict method to calculate the predicted matches.

In [ ]:
output = linker.inference.predict()

This generates the information in a DuckDB database. To look at these records, we can convert the output to a polars dataframe and sort by match probablity. You should note that the highest match_probability is quite low here.

In [ ]:
output_df = pl.DataFrame(output.as_pandas_dataframe()).sort(
    "match_probability", descending=True
)

When we look at the top matches where country names do not match exactly, we can see that the Jaro-Winkler similarity works for some of the cases we might hope it does.

In [ ]:
output_df.filter((pl.col("country_l") != pl.col("country_r")))

You results may be slightly different, but you should see some good results like "UNITED STATES" matching with "UNITED STATES OF AMERICA" and "BAHAMAS" matching with "BAHAMAS, THE". However, other results are not good like "IRELAND" matching with "ICELAND" and "NORTH MACEDONIA" matching with "NORTH AMERICA". We can improve our matches by adding more comparison fields. Specifically, we can add comparisons based on the receipts and expenditures numbers. We should expect the datasets to have similar numbers for these fields for the same countries and years. Here, we can build our own comparisons using predefined levels from splink. Each comparison is built as a set of levels where each level has a comparison function and a threshold.

In [ ]:
import splink.comparison_library as cl
import splink.comparison_level_library as cll
from splink import DuckDBAPI, Linker, SettingsCreator, block_on

settings = SettingsCreator(
    link_type="link_only",
    comparisons=[
        cl.JaroWinklerAtThresholds(
            "country",
            [0.9, 0.8, 0.7],
        ),
        cl.ExactMatch("year"),
        {
            "comparison_levels": [
                cll.NullLevel("receipts"),
                cll.PercentageDifferenceLevel("receipts", 0.05),
                cll.PercentageDifferenceLevel("receipts", 0.1),
                cll.PercentageDifferenceLevel("receipts", 0.2),
                cll.PercentageDifferenceLevel("receipts", 1.0),
            ],
        },
        {
            "comparison_levels": [
                cll.NullLevel("expenditures"),
                cll.PercentageDifferenceLevel("expenditures", 0.05),
                cll.PercentageDifferenceLevel("expenditures", 0.1),
                cll.PercentageDifferenceLevel("expenditures", 0.2),
                cll.PercentageDifferenceLevel("expenditures", 1.0),
            ],
        },
    ],
    blocking_rules_to_generate_predictions=[block_on("year")],
)

db_api = DuckDBAPI()
linker = Linker(
    [wb_w_ind, oecd_w_ind, unwto_w_ind],
    settings,
    db_api=db_api,
    input_table_aliases=["wb", "oecd", "unwto"],
)

In [ ]:
linker.training.estimate_u_using_random_sampling(max_pairs=1e7)
linker.training.estimate_parameters_using_expectation_maximisation(block_on("year"))
linker.training.estimate_parameters_using_expectation_maximisation(
    block_on("countryCodeAlpha"), block_on("countryCodeNum")
)
output = linker.inference.predict()

Here, we can see a visualization of the effect of the differnt parameter estimates.

In [ ]:
linker.visualisations.parameter_estimate_comparisons_chart()

In [ ]:
output_df = pl.DataFrame(output.as_pandas_dataframe()).sort(
    "match_probability", descending=True
)

In [ ]:
output_df.filter((pl.col("country_l") != pl.col("country_r"))).unique(
    subset=["country_l", "country_r"], keep="first", maintain_order=True
)

From the outputs, we can see that we are getting better matches by using the numeric values. For this dataset with a recent laptop, we can run this prediction quickly and transfer all of the results back, but many of the results are not very good, and we can use a thresold to filter them out before transferring them from the database. We can access the DuckDB database directly from polars using the `read_duckdb` method. Here, we will only select matches with a match probability greater than 0.5.

In [ ]:
query = f"SELECT * FROM {output.physical_name} WHERE match_probability > 0.5 ORDER BY match_probability DESC"
output_filtered = pl.read_database(
    query=query,
    connection=db_api._con,
)

We might be interested in those matches involving OECD countries since we manually matched them earlier.

In [ ]:
output_filtered.filter(
    ((pl.col("source_dataset_l") == "oecd") | (pl.col("source_dataset_r") == "oecd"))
    & (pl.col("country_l") != pl.col("country_r"))
).unique(subset=["country_l", "country_r"], keep="first", maintain_order=True)

Splink was able to match all six of these countries automatically using our rules!

### Data Fusion

Remember that our end goal is to fuse the three datasets together. When we have multiple records for the same country and year, we want to combine them into a single record. The first step is to use the information from entity resolution to make it clear which records are available from each dataset for the same country and year. We will start by concatenating the three datasets together and finding those rows where either one of the country codes is missing.

In [ ]:
all = pl.concat([unwto_w_ind, oecd_w_ind, wb_w_ind], how="diagonal")

In [ ]:
missing = all.filter(
    pl.col("countryCodeAlpha").is_null() | pl.col("countryCodeNum").is_null()
)

We want to find any codes that exist in the matched records.

In [ ]:
missing_l = missing.join(
    output_filtered.select(
        "source_dataset_l",
        "unique_id_l",
        pl.col("source_dataset_r").alias("source_dataset_other"),
        pl.col("unique_id_r").alias("unique_id_other"),
    ),
    left_on=["source_dataset", "unique_id"],
    right_on=["source_dataset_l", "unique_id_l"],
)
missing_r = missing.join(
    output_filtered.select(
        "source_dataset_r",
        "unique_id_r",
        pl.col("source_dataset_l").alias("source_dataset_other"),
        pl.col("unique_id_l").alias("unique_id_other"),
    ),
    left_on=["source_dataset", "unique_id"],
    right_on=["source_dataset_r", "unique_id_r"],
)
missing_matches = pl.concat([missing_l, missing_r], how="diagonal")

Next, we will join these other datasets to fill in the missing country codes.

In [ ]:
missing_fixes = (
    missing_matches.join(
        all.select("countryCodeAlpha", "countryCodeNum", "source_dataset", "unique_id"),
        left_on=["source_dataset_other", "unique_id_other"],
        right_on=["source_dataset", "unique_id"],
    )
    .with_columns(
        pl.coalesce(pl.col("countryCodeAlpha"), pl.col("countryCodeAlpha_right")),
        pl.coalesce(pl.col("countryCodeNum"), pl.col("countryCodeNum_right")),
    )
    .drop(
        "countryCodeNum_right",
        "countryCodeAlpha_right",
        "source_dataset_other",
        "unique_id_other",
    )
)

In [ ]:
all_fixed = pl.concat([missing_fixes, all]).unique(
    subset=["source_dataset", "unique_id"], maintain_order=True, keep="first"
)

### Finding Matching Rows

Now that we have filled in missing country codes, we can use this information to find rows that match across datasets. This is a  group_by operation on the codes and year.

In [ ]:
all_fixed.group_by("countryCodeNum", "countryCodeAlpha", "year").agg(
    [
        pl.col("country"),
        pl.col("receipts"),
        pl.col("expenditures"),
        pl.col("source_dataset"),
    ]
)

Now, we have to determine criteria to fuse the records together. We have a number of options:

* Random: Randomly select one of the records.
* Preferred Dataset: Choose records from a preferred dataset when available.
* Mode: Choose the most common value across records.

For numeric fields, we have additional options:

* Mean
* Median
* Max/Min

Which method to to use depends on the data and the use case. Here, we will use the mode for country names and the mean for receipts and expenditures.

In [ ]:
fused = all_fixed.group_by("countryCodeNum", "countryCodeAlpha", "year").agg(
    [
        pl.col("country").mode().first(),
        pl.col("receipts").mean(),
        pl.col("expenditures").mean(),
    ]
)

Finally, we might drop all data when there is neither receipts nor expenditures data.

In [ ]:
fused.filter(
    pl.col("receipts").is_not_null().and_(pl.col("expenditures").is_not_null())
).sort("year", "country").select(
    "country", "countryCodeAlpha", "countryCodeNum", "year", "receipts", "expenditures"
)

##### Exercise

Can you find any issues with the proposed fusion strategy? How could you diagnose this? Experiment with different fusion strategies for the fields. 